# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [32]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [33]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [34]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [35]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [36]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [37]:
# Creating a Keyspace 
try:
    session.execute(
        "CREATE KEYSPACE IF NOT EXISTS sparkify \
        WITH REPLICATION = \
        {'class': 'SimpleStrategy', 'replication_factor': 1}"
    )

except Exception as e:
    print(e)

#### Set Keyspace

In [38]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Using Pandas library to read event_datafile_new.csv file and displaying 

In [39]:
file = 'event_datafile_new.csv'
df = pd.read_csv(file)
df.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data
### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


In [9]:
#Creating Table required for query 1.
query_1 = "CREATE TABLE IF NOT EXISTS song_sessions "
query_1 = query_1 + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession))"

try:
    session.execute(query_1)
except Exception as e:
    print(e)

In [10]:
## INSERTING statements into the `query` variable
query_1 = "INSERT INTO song_sessions(sessionId, itemInSession, artist, song, length)"
query_1 = query_1 + "VALUES(%s, %s, %s, %s, %s)"
for i, row in df.iterrows():
    session.execute(query_1, (row['sessionId'], row['itemInSession'], row['artist'], row['song'], row['length']))

In [ ]:
#### verifying that the data have been inserted into each table

In [14]:
## SELECT statement to verify the data was entered into the table
rows = session.execute('SELECT * from song_sessions')
for i in range(2):
    print(rows[i])

Row(sessionid=23, iteminsession=0, artist='Regina Spektor', length=191.08526611328125, song='The Calculation (Album Version)')
Row(sessionid=23, iteminsession=1, artist='Octopus Project', length=250.95791625976562, song='All Of The Champs That Ever Lived')
Row(sessionid=23, iteminsession=2, artist='Tegan And Sara', length=180.06158447265625, song='So Jealous')
Row(sessionid=23, iteminsession=3, artist='Dragonette', length=153.39056396484375, song='Okay Dolores')
Row(sessionid=23, iteminsession=4, artist='Lil Wayne / Eminem', length=229.58975219726562, song='Drop The World')


In [16]:
## Performing required query
query_1_select = "SELECT artist, song, length FROM song_sessions WHERE sessionId = 338 AND itemInSession = 4"

try:
    rows = session.execute(query_1_select)
    for row in rows:
        print(row)
except Exception as e:
    print(e)
              

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [40]:
## Creating Tables required for Query 2
query2 = "CREATE TABLE IF NOT EXISTS \
    playlist_session"
query2 = query2 + "(userId int, sessionId int, itemInSession int, \
    artist text, song text, firstName text, lastName text, \
    PRIMARY KEY((userId, sessionId), itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)

In [41]:
## Inserting Data into playlist_session table
for index, row in df.iterrows():
    query2_insert = "INSERT INTO playlist_session \
             (userId, sessionId, itemInSession, \
             artist, song, firstName, lastName ) "
    query2_insert = query2_insert + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    session.execute(query2_insert, (row['userId'], row['sessionId'], 
                            row['itemInSession'], row['artist'], row['song'], row['firstName'], 
                            row['lastName']))

In [44]:
## Performing the reuired query 2.

query2_select = "SELECT artist, song, firstName, lastName FROM playlist_session \
                 WHERE userId = 10 AND sessionId = 182 ORDER BY itemInSession"

try:
    rows =session.execute(query2_select)
    for row in rows:
        print(row)
except Exception as e:
    print(e)
                 

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [24]:
## Creating table requried for Query 3
query3 = "CREATE TABLE IF NOT EXISTS user_songchoice_table "
query3 = query3 + "(song varchar, userId int, \
                 firstName varchar, lastName varchar, \
                 PRIMARY KEY (song, userId))"
try:
    session.execute(query3)
except Exception as e:
    print (e)

In [26]:
# Insert Data into user_songchoice_table
for index, row in df.iterrows():
    query = "INSERT INTO user_songchoice_table \
             (song, userId, \
             firstName, lastName) "
    query = query + "VALUES (%s, %s, %s, %s)"
    session.execute(query, (row['song'], row['userId'], 
                            row['firstName'], row['lastName']))

In [29]:
## Performing and displaying results for query 3
query3_select = "SELECT firstName, lastName from user_songchoice_table \
                 WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query3_select)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [30]:
## Droping all the tables before closing out the sessions
try:
    session.execute("DROP TABLE song_sessions")
    session.execute("DROP TABLE playlist_session")
    session.execute("DROP TABLE user_songchoice_table")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [31]:
session.shutdown()
cluster.shutdown()